<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/TE_spaCy_extract_noun_phrase_from_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Declare whether you are on Colab or local
colab = True

In [2]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')

  #set path to data in Google Drive
  data_path = "/content/drive/MyDrive/Colab Notebooks/TWSM Project/2022_Analytics Lab Student Projects/Data/All Topics"


  #install required packages
  #!pip install duckdb
  #!pip install -U sentence-transformers

Mounted at /content/drive


In [3]:
import pandas as pd
!pip install duckdb
import duckdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 27.2 MB/s 


In [4]:
# Load sentences
if colab==True:
  sentences = pd.read_csv(data_path+"/Fulltext of Corpus/sentences.csv")
else:
  sentences = pd.read_csv("../Data/FulltextofCorpus/sentences.csv")

In [5]:
sentences.head(5)

,article_id,sentence_id,para_id,sentence,section_nr,last_section_title,last_subsection_title,sentence_type,references,number_of_citations
0,1,1_0_1,1_0,,0,NaN,NaN,EMPTY,[],0
1,1,1_1_2,1_0,START_TITLE_TAG,0,NaN,NaN,TAG,[],0
2,1,1_2_22,1_0,Examining interdependence between product user...,0,NaN,NaN,PARAGRAPH,[],0
3,1,1_22_23,1_1,START_ABSTRACT_TAG,0,NaN,NaN,TAG,[],0
4,1,1_23_24,1_1,,0,Abstract,NaN,EMPTY,[],0


In [6]:
# Note that DBeaver must be closed, before you can run this command!
if colab==True:
  con = duckdb.connect(database=data_path+'/Extracted Information of Corpus/isrecon.duckdb', read_only=False)
else:
  con = duckdb.connect(database='../Data/ExtractedInformationofCorpus/isrecon.duckdb', read_only=False)

In [7]:
# Load first article as df
entities_sample_article = entities = con.execute("""
                       SELECT *
                       FROM entities e
                       WHERE e.article_id = 1""").fetchdf()

entities_sample_article.shape

(132, 46)

In [8]:
entities_sample_article.head()     # 1_1074_1096

,article_id,para_id,sentence_start,sentence_id,section_title,subsection_title,label,ent_id,level_1,level_2,...,attr_groups,attr_type,attr_tool,attr_datatype,attr_assumption,attr_double,attr_dsr,attr_related,attr_level,attr_stage
0,1,1_3,322,1_322_354,Introduction,NaN,TOPIC,knowledge repository,domain specific entity,IS topic,...,0,0,0,0,0,0,0,0,0,0
1,1,1_3,467,1_467_502,Introduction,NaN,TOPIC,digital platform,domain specific entity,IS topic,...,0,0,0,0,0,0,0,0,0,0
2,1,1_3,502,1_502_527,Introduction,NaN,TECHNOLOGY,Salesforce.com,domain specific entity,IS technology,...,0,0,0,0,0,0,0,0,0,0
3,1,1_4,530,1_530_551,Introduction,NaN,TOPIC,participatory design,domain specific entity,IS topic,...,0,0,0,0,0,0,0,1,0,0
4,1,1_5,861,1_861_886,Introduction,NaN,PARTICIPANTS,group participant,domain specific entity,study object,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# Import the table "entities" as a df
entities = con.execute("SELECT * FROM entities").fetchdf()

In [ ]:
#print(entities) 1_1074_1096

In [ ]:
#load entity extracion ruler
path_to_jsonl = (data_path+"/spaCy NER based on Ontology/all.jsonl")

In [ ]:
import spacy
from spacy.pipeline import EntityRuler

spacy_model = "en_core_web_sm"
nlp = spacy.load(spacy_model, disable=["ner"])
# nlp = spacy.load(spacy_model)
entity_detector = EntityRuler(nlp, validate=True)
path_to_jsonl =  data_path+"/spaCy NER based on Ontology/all.jsonl"
entity_detector.from_disk(path_to_jsonl)
# nlp.add_pipe(entity_detector, before="ner")
nlp.add_pipe(entity_detector)
# nlp.to_disk(nlp, "model/isrecon_spacy")
#return nlp

ValueError: ignored

In [114]:
pd.options.display.max_colwidth = 500
test_sentence = str(sentences.iloc[[58][:]])
print(test_sentence)

    article_id  sentence_id para_id  \
58           1  1_1074_1096     1_7   

                                                                                                                               sentence  \
58  Employee-generated content in OUC refers to the knowledge created and shared by employees across the community for product support.   

    section_nr last_section_title last_subsection_title sentence_type  \
58           1       Introduction                   NaN     PARAGRAPH   

   references  number_of_citations  
58         []                    0  


In [121]:
sample_ent = entities_sample_article.loc[entities_sample_article["sentence_id"]=="1_1074_1096"]
print(sample_ent)

Empty DataFrame
Columns: [article_id, para_id, sentence_start, sentence_id, section_title, subsection_title, label, ent_id, level_1, level_2, level_3, level_4, level_5, level_6, level_7, level_8, level_9, level_10, paired, temporal, role, setting, groups, type, tool, datatype, assumption, double, dsr, related, level, stage, attr_paired, attr_temporal, attr_role, attr_setting, attr_groups, attr_type, attr_tool, attr_datatype, attr_assumption, attr_double, attr_dsr, attr_related, attr_level, attr_stage]
Index: []

[0 rows x 46 columns]


In [119]:
sample_ent = entities_sample_article[(entities_sample_article["sentence_id"]=="1_1074_1096")]
print(sample_ent)

Empty DataFrame
Columns: [article_id, para_id, sentence_start, sentence_id, section_title, subsection_title, label, ent_id, level_1, level_2, level_3, level_4, level_5, level_6, level_7, level_8, level_9, level_10, paired, temporal, role, setting, groups, type, tool, datatype, assumption, double, dsr, related, level, stage, attr_paired, attr_temporal, attr_role, attr_setting, attr_groups, attr_type, attr_tool, attr_datatype, attr_assumption, attr_double, attr_dsr, attr_related, attr_level, attr_stage]
Index: []

[0 rows x 46 columns]


In [45]:
doc = nlp(test_sentence)
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

                                                                                                                                                                                                                                                                SPACE _SP dep LONG False False
sentence sentence NOUN NN nmod xxxx True False

 
 SPACE _SP dep 
 False False
58 58 NUM CD nummod dd False False
    SPACE _SP dep   False False
Employee employee NOUN NN npadvmod Xxxxx True False
- - PUNCT HYPH punct - False False
generated generate VERB VBN amod xxxx True False
content content NOUN NN nsubj xxxx True False
in in ADP IN prep xx True True
OUC OUC PROPN NNP pobj XXX True False
refers refer VERB VBZ ROOT xxxx True False
to to ADP IN prep xx True True
the the DET DT det xxx True True
knowledge knowledge NOUN NN pobj xxxx True False
created create VERB VBN acl xxxx True False
and and CCONJ CC cc xxx True True
shared share VERB VBN conj xxxx True False
by by ADP IN agent xx True True
employe

In [ ]:
spacy.explain("dobj")

'direct object'

In [ ]:
spacy.explain("VBG")

'verb, gerund or present participle'

In [ ]:
from spacy import displacy

In [36]:
displacy.render(doc, style="dep", jupyter=True)

NameError: ignored

### Only extract nouns

In [ ]:
nouns = []

for token in doc:
  if token.pos_ == "NOUN":
    nouns.append(token)

print(nouns)

[sentence, contrary, knowledge, contribution, reader, readers, knowledge, contribution, source, content, user, community]


### Noun chunks

Noun chunks are “base noun phrases” – flat phrases that have a noun as their head: noun plus the words describing the noun

In [46]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

                                                                                                                               sentence
58  Employee-generated content content nsubj refers
OUC OUC pobj in
the knowledge knowledge pobj to
employees employees pobj by
the community community pobj across
product support support pobj for


The participation of employees thereby holds the potential to help sustain and realize value from the community

### Navigating the parse tree

terms **head and child** describe the words connected by a single arc in the dependency tree. The term dep is used for the arc label, which describes the **type of syntactic relation** that connects the child to the head. As with other attributes, the value of .dep is a hash value. You can get the string value with .dep_.

In [ ]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

                                                                                                                                                                                                                                          sentence NOUN []
sentence ROOT sentence NOUN [                                                                                                                                                                                                                                        , 
, 15]

  sentence NOUN []
15 nummod sentence NOUN [ ]
   15 NUM []
On prep sparks VERB [contrary]
the det contrary NOUN []
contrary pobj On ADP [the]
, punct sparks VERB []
reading nsubj sparks VERB []
sparks ROOT sparks VERB [On, ,, reading, contribution, and, having, .]
additional amod contribution NOUN []
knowledge compound contribution NOUN []
contribution dobj sparks VERB [additional, knowledge, by]
by prep contribution NOUN [reader]
the det reader NOUN []
reader pobj by ADP 

### Adding Entity Extraction Rules

EntityRuler - uses token and token attribues

Matcher-Patterns - Lists of dictionaries, one per Token

## return a table with sentence ID + Entity
